# MongoDB Lecture 02:

**`with Mr Fugu Data Science`**


# (◕‿◕✿)

[Youtube](https://www.youtube.com/channel/UCbni-TDI-Ub8VlGaP8HLTNw/) | [Github](https://github.com/MrFuguDataScience)


# Objective & Outcome:

+ Create Updates
    + Deal with Nested data
        + Indexing
    
            + Aggregates

In [190]:
import faker # Crete fake People
import pandas as pd # Dataframes
import random # random number generation
import requests  # grab web-page
from bs4 import BeautifulSoup as bsopa  #parse web-page 
import re # regex parsing
import pymongo as pym

In [173]:
"""
I want to generate skills of employees ranging from 1:(1-len(skills) list).
I am generating 500 entries, because I will populate 500 fake prospective employees
to use in my Fake Employment Recruiter Database today.

"""

skills=['Python','Java','SQL','MongoDB','R','C++','Spark','TensorFlow','skLearn']


def create_rndm_sklls(skills):
    random.seed(300) # USE TO KEEP DATA CONSISTENT
    randomlist = []
    for i in range(0,500):
        n = random.randint(1,len(skills))
        randomlist.append(n)
    # print(randomlist)

    lst_skills_per_person=[]
    for i in randomlist:
        lst_skills_per_person.extend([random.sample(skills, i)])
    lst_skills_per_person
    return lst_skills_per_person

create_rndm_sklls(skills)[:5]

[['skLearn', 'Java', 'R', 'SQL', 'Spark', 'C++'],
 ['TensorFlow', 'R', 'Spark', 'MongoDB', 'C++', 'SQL'],
 ['Spark'],
 ['skLearn', 'SQL', 'R', 'Spark', 'Java', 'C++', 'Python', 'TensorFlow'],
 ['TensorFlow', 'C++', 'SQL', 'Java', 'R', 'MongoDB']]

In [174]:
fake_=faker.Faker()
fake_.seed(413)
fake_name=[]
for i in range(500):
    if i not in fake_name:
        fake_name.append(fake_.name().split(' '))
        
fake_name[:5]


[['Margaret', 'Mcdonald'],
 ['Michael', 'Carter'],
 ['Brenda', 'Tyler'],
 ['Joseph', 'King'],
 ['Laura', 'Webb']]

In [175]:
response=requests.get("https://www.softschools.com/social_studies/state_abbreviations/")

response.status_code # checking that the request was (ok=200)

soupie=bsopa(response.text,'lxml')  # converting format

html_source=soupie.find('div',{"class":"middlecol"}) # html source code

# tags=html_source.find('a').find_all('href')


In [176]:
# Get US State Abreviations:

table = html_source.find('table' ,attrs={'class':'colorBgGreen'})
first_td = table.find('tr')
text=first_td.renderContents()
text_=text.decode("utf-8") # convert bytes to string
jj=text_.split('\n')

state_td_tags=[]
for i in jj:
    if re.findall(r'\b[A-Z]{2}\b',i):
#         trr.append(i.replace('<td>','').replace('</td>',''))
        state_td_tags.append(i.replace('<td>','').replace('</td>',''))
    
state_abbr=state_td_tags[:-2] #the last two lines i didn't need US Territories
# state_abbr


In [177]:
# 500 random US states using, (N CHOOSE K):

state_rnd=random.choices(state_abbr,k=500)

In [178]:
# Relocation: (Employee willing to relocate?)
willing_to_relocate=['yes','no','maybe']
relocation_=random.choices(willing_to_relocate,k=500)

In [233]:
# Nested Data:

specialty=['Machine Learning','Data Visualization','Database','Statistics']
experience_level=['Junior','Mid','Senior']

spec_=random.choices(specialty,k=500)
exper_=random.choices(experience_level,k=500)
sp=['specialty']*500
exp=['experience']*500
my_list=list(zip(sp,spec_))
my_list2=list(zip(exp,exper_))


dict_specialty=[{key: value} for (key, value) in my_list]

dict_exper=[{key: value} for (key, value) in my_list2]

# This will be our tuple of dictionary pairs:
spec_exp=list(zip(dict_specialty,dict_exper))


In [234]:
# Generating, the full dataset

first_name=[]
last_name=[]
for i in fake_name:
    first_name.append(i[0])
    last_name.append(i[1])


nested_ppl=list(zip(first_name,last_name,create_rndm_sklls(skills),
                    state_rnd,spec_exp,relocation_))
nested_ppl[:2]

[('Margaret',
  'Mcdonald',
  ['skLearn', 'Java', 'R', 'SQL', 'Spark', 'C++'],
  'AL',
  ({'specialty': 'Database'}, {'experience': 'Junior'}),
  'no'),
 ('Michael',
  'Carter',
  ['TensorFlow', 'R', 'Spark', 'MongoDB', 'C++', 'SQL'],
  'AR',
  ({'specialty': 'Data Visualization'}, {'experience': 'Junior'}),
  'yes')]

In [235]:
cols=['first_name','last_name','skills','state','specialty_exper','relocation']
job_seekers_=pd.DataFrame(nested_ppl,columns=cols,)

pd.DataFrame.to_csv(job_seekers_,'job_seekers.csv')
job_seekers_.head()

,first_name,last_name,skills,state,specialty_exper,relocation
0,Margaret,Mcdonald,"[skLearn, Java, R, SQL, Spark, C++]",AL,"({'specialty': 'Database'}, {'experience': 'Ju...",no
1,Michael,Carter,"[TensorFlow, R, Spark, MongoDB, C++, SQL]",AR,"({'specialty': 'Data Visualization'}, {'experi...",yes
2,Brenda,Tyler,[Spark],UT,"({'specialty': 'Database'}, {'experience': 'Mi...",no
3,Joseph,King,"[skLearn, SQL, R, Spark, Java, C++, Python, Te...",FL,"({'specialty': 'Statistics'}, {'experience': '...",maybe
4,Laura,Webb,"[TensorFlow, C++, SQL, Java, R, MongoDB]",WY,"({'specialty': 'Machine Learning'}, {'experien...",maybe


# MongoDB Lecture 02:

**`with Mr Fugu Data Science`**


# (◕‿◕✿)

[Youtube](https://www.youtube.com/channel/UCbni-TDI-Ub8VlGaP8HLTNw/) | [Github](https://github.com/MrFuguDataScience)


# Objective & Outcome:

+ Create Updates
    + Deal with Nested data
        + Indexing
    
            + Aggregates

# Creating Actual `Nested Dictionary (JSON)` format for MongoDB

In [236]:
candidates_=list(zip(first_name,last_name,create_rndm_sklls(skills),
                    state_rnd,spec_,exper_,relocation_))
cols_=['first_name','last_name','skills','state','specialty','experience','relocation']

un_nested_candidates=pd.DataFrame(candidates_,columns=cols_)

In [237]:


def df_to_nested_json(df, candidate, lat='state'):
    _json = {'features':[]}
    
    for _, row in df.iterrows():
        feature = {'candidate':{}}
#                    'specialty_experience':{}}
#         feature['specialty_experience']['state'] = [row[lat]]
#         feature['specialty_experience']['skills'] = [row[lon]
        for prop in candidate:
            feature['candidate'][prop] = row[prop]
        _json['features'].append(feature)
    return _json

#-------------------------

cols=un_nested_candidates.columns

people_json=df_to_nested_json(un_nested_candidates, cols)

people_json['features'][:2]

[{'candidate': {'first_name': 'Margaret',
   'last_name': 'Mcdonald',
   'skills': ['skLearn', 'Java', 'R', 'SQL', 'Spark', 'C++'],
   'state': 'AL',
   'specialty': 'Database',
   'experience': 'Junior',
   'relocation': 'no'}},
 {'candidate': {'first_name': 'Michael',
   'last_name': 'Carter',
   'skills': ['TensorFlow', 'R', 'Spark', 'MongoDB', 'C++', 'SQL'],
   'state': 'AR',
   'specialty': 'Data Visualization',
   'experience': 'Junior',
   'relocation': 'yes'}}]

In [240]:
# Making a Connection to MongoClient
client = pym.MongoClient('mongodb://localhost:27017/')

# CREATING A DATABASE:
db = client["berkeley"]

# CREATING A COLLECTION (*AKA* TABLE):
recruiter_clients= db["recruiter_clients"]

In [241]:
# SENDING THE LIST OF DICTIONARIES TO MONGO via PyMongo:

db.recruiter_clients.insert_many(people_json['features'])

# `Now Time to Start MongoDB:`

`_________________________________`



# nested data:

+ Think *SUB*documents, where you are using: out_key.inner_key to start entering the nested documents.

**Example of our dataset ( 2 entries )**:

`{[{'candidate': {'first_name': 'Margaret',
   'last_name': 'Mcdonald',
   'skills': ['skLearn', 'Java', 'R', 'SQL', 'Spark', 'C++'],
   'state': 'AL',
   'specialty': 'Machine Learning',
   'experience': 'Mid',
   'relocation': 'no'}},
 {'candidate': {'first_name': 'Michael',
   'last_name': 'Carter',
   'skills': ['TensorFlow', 'R', 'Spark', 'MongoDB', 'C++', 'SQL'],
   'state': 'AR',
   'specialty': 'Machine Learning',
   'experience': 'Junior',
   'relocation': 'yes'}}]`
 
  
  
`____________________________________`

**`Query Nested Portion`**: use the (*dot*) operator, think method like Java or Python

# Nested Queries:


`Find Only people with skills in Spark`
  
**`db.recruiter_clients.find({'candidate.skills':['Spark']},{_id:0});`**

`Find all people who have at least Python`

**`db.recruiter_clients.find({'candidate.skills':{`$all`:['Python']}},{_id:0});`**


`Find all that (do not) have certain skills`:

**`db.recruiter_clients.find({'candidate.skills':{`$nin`:['Python']}},{_id:0});`**

`People who live in California and don't want to move`:

**`db.recruiter_clients.find({'candidate.state':'CA','candidate.relocation':'no'});`**

`Find People in California OR Florida`:

**`db.recruiter_clients.find({$or :[{'candidate.state':'CA'},{'candidate.state':'FL'}]});`**

`-------------------------------------`

[Mongo embeded queries doc](https://docs.mongodb.com/manual/tutorial/query-array-of-documents/)

`__________________________________________`

# Update:

+ Requires two parameters: `(citeria,operation)`

*`Criteria`:* think of objects that will be used with `find( )`

*`Operation:`* either a modifier operation, or an object that will match the fields for the document you will replace.


[update_mongo doc](https://docs.mongodb.com/manual/reference/method/db.collection.update/#update-method-examples) | [update operators mongo](https://docs.mongodb.com/manual/reference/operator/update/)

| Name           	| Description                                                                                                       	|
|----------------	|-------------------------------------------------------------------------------------------------------------------	|
| **`$currentDate`** 	| set as *timestamp* or *Date*                                                                                      	|
| **`$inc`**        	| increments                                                                                                        	|
|**`$min`**         	| only changes if the new min. value is < current value                                                             	|
|**`$max`**         	| only changes if new  max is > current value                                                                       	|
| **`$mul`**         	| multiplies field by specified value                                                                               	|
| **`$rename`**      	| renames field                                                                                                     	|
| **`$set`**        	| will set, the value to field                                                                                      	|
| **`$setOnInsert`** 	| **1.)** will set value if it inserts a field into document. **2.)** Doesn't effect/modify existing documents with update 	|
| **`$unset`**       	| removes the field from document                                                                                   	|

`_________________________________`

*This is not a full list and adapted from MongoDB* documentation*

# General Format for update:


`{
  <update operator>: { <field1>: <value1>, ... },
  <update operator>: { <field2>: <value2>, ... },
  ...
}`





`______________________`

# Update Queries:

`Update a Last_Name for 1 perosn using `$set` operator`:

**`db.recruiter_clients.updateOne({_id : ObjectId("5eb5cc5b47c72ccce7539d9f")},{$set:{'candidate.last_name':'Stoute'}, $currentDate: { lastModified: true }});`**

* you don't need to add the *`lastModified`*

`Update the last person and ADD a new skill with `$push` operator`

**`db.recruiter_clients.updateOne({_id : ObjectId("5eb5cc5b47c72ccce7539d9f")},{$push:{'candidate.skills':'Python'}});`**



`___________________________`


# Indexing:

+ When you create a `document`, mongo will generate an `index` for the *`_id`* attribute. You can find this by: *system.indexes* for your collection such as **`db.your_collection_name.getIndexes()`**
    + This will return an array within your collection, containing indexex

+ *`Without Indexes, mongo would have to search the whole document in your collection!`*
    + Indexes, limits the number of searches (saving you time), because they save you from unnecessary read operations. 
    
    + *`An Index, will store a small portion of your collection to traverse. It will either: a specific field or multiple fields and then order them. Because, they are ordered you are able to run quieres based on range or match by equality for example.`* 
    
+ *A few types of Indexing: `B-Tree, spherical GeoJSON, two-dimensional`*

[.getIndex( ) info](https://docs.mongodb.com/manual/reference/method/db.collection.getIndexes/) | [Index specifics](https://docs.mongodb.com/manual/indexes/)


`A few side notes about Indexes`:

+ They can be more costly than if you were using PostgreSQL (since Mongo is schema-less) 
+ Use them when there are non-peak times if possible



# Create Index, based on Text keywords:

`Setup an Index, where we use two fields "skills and specialty" where we search text for specific values`

**`db.recruiter_clients.createIndex({'candidate.skills':'text','candidate.specialty':'text'},{name:'text_search'});`**

`Query our Index, find any of the text words for "Python or Machine Learning"`:

**`db.recruiter_clients.find({$text: {$search:'Python, Machine Learning'}});`**


`Manage/Find Indexes you may have`:

**`db.recruiter_clients.getIndexes()`**

**`Drop All Indexes from a specific collection: if you don't need them`**:

`db.runCommand( { dropIndexes: "recruiter_clients", index: "*" } )`


**`Drop multiple Indexes by Name of Index`**:

`db.runCommand( { dropIndexes: "collection", index: [ "Index_name01", "Index_name02" ] }
)`

**`Drop The current Index we made by the Name 'text_search'`**:

`db.recruiter_clients.dropIndex('text_search');`

`_________________________________`

**`Alternate Text Index with Wildcard, able to search unstructured data and look for specific queries through all fields that are strings`**:

`db.recruiter_clients.createIndex( { "$**": "text" },{name:'text_s'} )`

**`To drop this Index`**:

`db.recruiter_clients.dropIndex('text_s' )`


`----------------------------------------------------`

# Aggregate:

+ Take in some form of computation and return a result. There are 3 types of aggregation:

1.) *`Aggregation Pipline:`* basically, a multi-stage function

+ Can use indexing to improve performance
+ Can be used with *sharding*
+ you can apply filters, grouping, averaging of a specific field/s are just a few

 * A few examples in the pipline: `$group, $match, $sort` 
 * You can do Chaining just like PostgreSQL such as: `WHERE, GROUP BY, ORDER BY `

2.)*`map reduce:`* less effective than a pipline; but, it is highly flexible since its *Javascript*
+ *`Map`*: takes data splits it, does operations
+ *` Reduce`*: recombines data

3.) *`single purpose aggregation:`* will aggregate based on a single collection

`-------------------------`

+ A few single purpose types: `Count( ),aggregate( ), distinct( )`


[Aggregate info](https://docs.mongodb.com/manual/aggregation/) | 

**`Find the Number of Skills each person has`**

` db.recruiter_clients.aggregate([{$project:{'candidate.first_name':1,'candidate.last_name':1,numberOfSkills:{$cond:{if:{$isArray:'$candidate.skills'},then:{$size:'$candidate.skills'},else:'NA'}}}}])`

**`Find all matches in a string where the specialty is Machine Learning`**

`db.recruiter_clients.aggregate( [ { $match: { 'candidate.specialty': 'Machine Learning'}}])`

_____________------------____________------------

# Like

# SUBscribe

+ THANKS FOR WATCHING AND SEE YOU IN THE NEXT VIDEO!

# Citations:

https://www.red-gate.com/simple-talk/blogs/enjoying-joins-in-mongodb/ (MongoDB Join Info)

https://medium.com/dbkoda/coding-efficient-mongodb-joins-97fe0627751a

https://www.researchgate.net/publication/335873099_Performance_analysis_of_Not_Only_SQL_semi-stream_join_using_MongoDB_for_real-time_data_warehousing

https://stackoverflow.com/questions/49580996/why-do-my-lists-become-strings-after-saving-to-csv-and-re-opening-python

https://stackoverflow.com/questions/57238899/function-to-create-nested-dictionary-from-lists

https://medium.com/@paulrohan/aggregation-in-mongodb-8195c8624337